In [46]:
import pandas
import subprocess
import json
import re

base_url = 'https://info.gesundheitsministerium.at/data/'

def getVariableFrom(var_name, string_js):
    pattern = re.compile('^var\s+'+var_name+'\s*=\s*(.*);.*$', re.M)
    match = pattern.search(string_js)
    if not match:
        raise Exception("Failed to get {}!".format(var_name))
    else:
        return json.loads(match.group(1))



simpleJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'SimpleData.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
infected = getVariableFrom('Erkrankungen', simpleJs)
hospitalised = getVariableFrom('hospitalisiert', simpleJs)
intensive_care = getVariableFrom('Intensivstation', simpleJs)
last_update = getVariableFrom('LetzteAktualisierung', simpleJs)
#print("erkrankungen = {}".format(erkrankungen))
#print("hospitalisiert = {}".format(hospitalisiert))
#print("intensivstation = {}".format(intensivstation))
#print("letzteAktualisierung = {}".format(letzteAktualisierung))

districtsJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'Bezirke.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
bezirke = getVariableFrom('dpBezirke', districtsJs)
df_districts = pandas.DataFrame.from_records(bezirke, 'label')
df_districts.index.name = "Bezirk"
df_districts.rename(columns={'y': "Infected (confirmed)"}, inplace=True)
#print("df_districts:\n{}".format(df_districts))

trendJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'Trend.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
trend = getVariableFrom('dpTrend', trendJs)
df_trend = pandas.DataFrame.from_records(trend, 'label')
df_trend.index.name = "Date"
df_trend.rename(columns={'y': "Infected (confirmed)"}, inplace=True)
df_trend.rename(index=lambda idx: idx+".2020", inplace=True)
df_trend.index = pandas.to_datetime(df_trend.index, format="%d.%m.%Y")
#print("trend:\n{}".format(df_trend))


statesJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'Bundesland.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
bundesland = getVariableFrom('dpBundesland', statesJs)
df_state = pandas.DataFrame.from_records(bundesland, 'label')
df_state.index.name = "State"
df_state.rename(columns={'y': "Infected (confirmed)"},
                index={'Bgld':'Burgenland',
                       'Ktn':'Kärnten',
                       'NÖ':'Niederösterreich',
                       'OÖ':'Oberösterreich',
                       'Sbg':'Salzburg',
                       'Stmk':'Steiermark',
                       'T':'Tirol',
                       'Vbg':'Vorarlberg',
                       'W':'Wien'},
                inplace=True)
#rint("df_state:\n{}".format(df_state))

genderJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'Geschlechtsverteilung.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
geschlechtsverteilung = getVariableFrom('dpGeschlechtsverteilung', genderJs)
df_sex = pandas.DataFrame.from_records(geschlechtsverteilung, 'label')
df_sex.index.name = "Sex"
df_sex.rename(columns={'y': "Infected (confirmed)"},
                index={'männlich':'Male', 'weiblich':'Female'},
                inplace=True)
#print("df_sex:\n{}".format(df_sex))

ageDistJs = subprocess.run(['wget', '-O', '-', '-q', base_url+'Altersverteilung.js'], stdout=subprocess.PIPE).stdout.decode('utf-8')
altersverteilung = getVariableFrom('dpAltersverteilung', ageDistJs)
df_age = pandas.DataFrame.from_records(altersverteilung, 'label')
df_age.index.name = "Age"
df_age.rename(columns={'y': "Infected (confirmed)"}, inplace=True)
#print("df_age:\n{}".format(df_age))

df_age:
       Infected (confirmed)
Age                        
<5                       12
5-14                     72
15-24                   432
25-34                   741
35-44                   677
45-54                  1000
55-64                   731
>64                     788
